In [2]:
import numpy as np
import pandas as pd
import csv
import sys
import os

import matplotlib.pyplot as plt    
    
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
import warnings
import pickle

max_iters = 501


def svm(x,y,filename):

   # Model output file name
   file = (os.path.splitext(filename))[0]
   fname = './models/svm_' + file +'/'

   # File for writing precision,recall, f-measure scores for fraud transactions
   f = open('./prf/svm_'+ file + '_prf' +'.txt' ,'w')
   f.write('precision,recall,f-score \n')

   # Stratified sampling based on Y
   X_train, X_test, y_train, y_test = train_test_split(x, y,stratify=y , test_size=0.30, random_state=42)

   # Create 15% validation set and 15% test set split
   X_val, X_test, y_val, y_test = train_test_split(X_test, y_test,stratify=y_test , test_size=0.50, random_state=42)

   #Iterations
   it = 1
   
   # Run training algorithm for multiple class weights
   while it < max_iters:
       cw = {}
       cw[0] = 1
       cw[1] = it
       # Train
       print('**************************************')
       print("Iteration number  " , it)
       svm = LinearSVC(class_weight = cw, dual = False ,tol=1e-05,max_iter = 1000)
       print('Class weights ', cw)
       svm.fit(X_train,y_train)

       # Save trained model to disk
       name = fname + str(cw[1]) + '.sav'
       pickle.dump(svm, open(name, 'wb'))

       #Predict on validation data
       y_val_pred = svm.predict(X_val)
       print('Performance on validation data - Confusion matrix')
       print(confusion_matrix(y_val,y_val_pred))
   
       precision,recall,fscore,support=score(y_val,y_val_pred,average=None)
       print('Precision, Recall, F-score, Support on validation data' )
       print("Precision" , precision)
       print("Recall" , recall)
       print("F-score" , fscore)
       print("Support" , support)

       p1 = precision[1]
       r1 = recall[1]
       f1 = fscore[1]

       f.write(str(p1) +','+ str(r1) + ',' + str(f1) + '\n')    
       it += 1

   f.close()

def run():
   filename = 'm2.csv'
   df = pd.read_csv(filename, usecols = [9,10,11,12,13,14] , header = 0)
   
   results = list(map(int, df['Anomaly'])) 
   print('Number of fraudulent transactions ' , sum(results))

   features = ['Airfare', 'Lodging', 'Meals', 'Other_Transportation', 'Other_Expenses']
   targets = ['Anomaly']

   # Separating out the features and target variables
   x = df.loc[:, features].values
   y = df.loc[:, targets].values
    
   scaler = preprocessing.StandardScaler()
   x = scaler.fit_transform(x) 

   y  = [i for j in y for i in j]
   
   #Ignore warnings
   warnings.filterwarnings("ignore", category=FutureWarning)

   print("**************** SVM *******************")
   svm(x,y,filename)
  
run()

Number of fraudulent transactions  4116
**************** SVM *******************
**************************************
Iteration number   1
Class weights  {0: 1, 1: 1}
Performance on validation data - Confusion matrix
[[335007      0]
 [   360    258]]
Precision, Recall, F-score, Support on validation data
Precision [0.99892655 1.        ]
Recall [1.         0.41747573]
F-score [0.99946299 0.5890411 ]
Support [335007    618]
**************************************
Iteration number   2
Class weights  {0: 1, 1: 2}
Performance on validation data - Confusion matrix
[[334989     18]
 [   331    287]]
Precision, Recall, F-score, Support on validation data
Precision [0.99901288 0.94098361]
Recall [0.99994627 0.46440129]
F-score [0.99947936 0.62188516]
Support [335007    618]
**************************************
Iteration number   3
Class weights  {0: 1, 1: 3}
Performance on validation data - Confusion matrix
[[334971     36]
 [   312    306]]
Precision, Recall, F-score, Support on validatio

Performance on validation data - Confusion matrix
[[334376    631]
 [   213    405]]
Precision, Recall, F-score, Support on validation data
Precision [0.9993634  0.39092664]
Recall [0.99811646 0.65533981]
F-score [0.99873954 0.48972189]
Support [335007    618]
**************************************
Iteration number   25
Class weights  {0: 1, 1: 25}
Performance on validation data - Confusion matrix
[[334343    664]
 [   208    410]]
Precision, Recall, F-score, Support on validation data
Precision [0.99937827 0.38175047]
Recall [0.99801795 0.66343042]
F-score [0.99869765 0.48463357]
Support [335007    618]
**************************************
Iteration number   26
Class weights  {0: 1, 1: 26}
Performance on validation data - Confusion matrix
[[334321    686]
 [   206    412]]
Precision, Recall, F-score, Support on validation data
Precision [0.99938421 0.37522769]
Recall [0.99795228 0.66666667]
F-score [0.99866773 0.48018648]
Support [335007    618]
*************************************

Performance on validation data - Confusion matrix
[[333736   1271]
 [   172    446]]
Precision, Recall, F-score, Support on validation data
Precision [0.99948489 0.25975539]
Recall [0.99620605 0.72168285]
F-score [0.99784278 0.38201285]
Support [335007    618]
**************************************
Iteration number   49
Class weights  {0: 1, 1: 49}
Performance on validation data - Confusion matrix
[[333715   1292]
 [   172    446]]
Precision, Recall, F-score, Support on validation data
Precision [0.99948486 0.2566168 ]
Recall [0.99614336 0.72168285]
F-score [0.99781131 0.37860781]
Support [335007    618]
**************************************
Iteration number   50
Class weights  {0: 1, 1: 50}
Performance on validation data - Confusion matrix
[[333690   1317]
 [   170    448]]
Precision, Recall, F-score, Support on validation data
Precision [0.9994908  0.25382436]
Recall [0.99606874 0.72491909]
F-score [0.99777684 0.37599664]
Support [335007    618]
*************************************

Performance on validation data - Confusion matrix
[[333174   1833]
 [   146    472]]
Precision, Recall, F-score, Support on validation data
Precision [0.99956198 0.20477223]
Recall [0.99452847 0.76375405]
F-score [0.99703887 0.32295587]
Support [335007    618]
**************************************
Iteration number   73
Class weights  {0: 1, 1: 73}
Performance on validation data - Confusion matrix
[[333144   1863]
 [   145    473]]
Precision, Recall, F-score, Support on validation data
Precision [0.99956494 0.20248288]
Recall [0.99443892 0.76537217]
F-score [0.99699534 0.32024374]
Support [335007    618]
**************************************
Iteration number   74
Class weights  {0: 1, 1: 74}
Performance on validation data - Confusion matrix
[[333119   1888]
 [   144    474]]
Precision, Recall, F-score, Support on validation data
Precision [0.99956791 0.20067739]
Recall [0.9943643  0.76699029]
F-score [0.99695931 0.31812081]
Support [335007    618]
*************************************

Performance on validation data - Confusion matrix
[[332608   2399]
 [   131    487]]
Precision, Recall, F-score, Support on validation data
Precision [0.9996063  0.16874567]
Recall [0.99283896 0.78802589]
F-score [0.99621113 0.27796804]
Support [335007    618]
**************************************
Iteration number   97
Class weights  {0: 1, 1: 97}
Performance on validation data - Confusion matrix
[[332586   2421]
 [   131    487]]
Precision, Recall, F-score, Support on validation data
Precision [0.99960627 0.16746905]
Recall [0.99277329 0.78802589]
F-score [0.99617806 0.27623369]
Support [335007    618]
**************************************
Iteration number   98
Class weights  {0: 1, 1: 98}
Performance on validation data - Confusion matrix
[[332565   2442]
 [   130    488]]
Precision, Recall, F-score, Support on validation data
Precision [0.99960925 0.1665529 ]
Recall [0.9927106  0.78964401]
F-score [0.99614798 0.27508455]
Support [335007    618]
*************************************

...

Precision, Recall, F-score, Support on validation data
Precision [0.99996313 0.05956945]
Recall [0.97144239 0.98058252]
F-score [0.98549645 0.11231582]
Support [335007    618]
**************************************
Iteration number   402
Class weights  {0: 1, 1: 402}
Performance on validation data - Confusion matrix
[[325418   9589]
 [    12    606]]
Precision, Recall, F-score, Support on validation data
Precision [0.99996313 0.0594409 ]
Recall [0.97137672 0.98058252]
F-score [0.98546266 0.1120873 ]
Support [335007    618]
**************************************
Iteration number   403
Class weights  {0: 1, 1: 403}
Performance on validation data - Confusion matrix
[[325390   9617]
 [    11    607]]
Precision, Recall, F-score, Support on validation data
Precision [0.9999662  0.05937011]
Recall [0.97129314 0.98220065]
F-score [0.98542113 0.11197196]
Support [335007    618]
**************************************
Iteration number   404
Class weights  {0: 1, 1: 404}
Performance on validation 

Performance on validation data - Confusion matrix
[[324373  10634]
 [     9    609]]
Precision, Recall, F-score, Support on validation data
Precision [0.99997225 0.05416704]
Recall [0.96825738 0.98543689]
F-score [0.9838593  0.10268949]
Support [335007    618]
**************************************
Iteration number   426
Class weights  {0: 1, 1: 426}
Performance on validation data - Confusion matrix
[[324325  10682]
 [     9    609]]
Precision, Recall, F-score, Support on validation data
Precision [0.99997225 0.05393676]
Recall [0.9681141  0.98543689]
F-score [0.98378533 0.10227559]
Support [335007    618]
**************************************
Iteration number   427
Class weights  {0: 1, 1: 427}
Performance on validation data - Confusion matrix
[[324272  10735]
 [     9    609]]
Precision, Recall, F-score, Support on validation data
Precision [0.99997225 0.05368477]
Recall [0.96795589 0.98543689]
F-score [0.98370363 0.10182244]
Support [335007    618]
*********************************

Precision, Recall, F-score, Support on validation data
Precision [0.99997834 0.04895049]
Recall [0.96456492 0.98867314]
F-score [0.98195244 0.09328244]
Support [335007    618]
**************************************
Iteration number   449
Class weights  {0: 1, 1: 449}
Performance on validation data - Confusion matrix
[[323103  11904]
 [     7    611]]
Precision, Recall, F-score, Support on validation data
Precision [0.99997834 0.04882141]
Recall [0.96446641 0.98867314]
F-score [0.98190139 0.09304805]
Support [335007    618]
**************************************
Iteration number   450
Class weights  {0: 1, 1: 450}
Performance on validation data - Confusion matrix
[[323057  11950]
 [     7    611]]
Precision, Recall, F-score, Support on validation data
Precision [0.99997833 0.04864262]
Recall [0.9643291  0.98867314]
F-score [0.98183023 0.09272327]
Support [335007    618]
**************************************
Iteration number   451
Class weights  {0: 1, 1: 451}
Performance on validation 

Performance on validation data - Confusion matrix
[[321876  13131]
 [     7    611]]
Precision, Recall, F-score, Support on validation data
Precision [0.99997825 0.04446223]
Recall [0.9608038  0.98867314]
F-score [0.9799997  0.08509749]
Support [335007    618]
**************************************
Iteration number   473
Class weights  {0: 1, 1: 473}
Performance on validation data - Confusion matrix
[[321822  13185]
 [     7    611]]
Precision, Recall, F-score, Support on validation data
Precision [0.99997825 0.0442882 ]
Recall [0.96064261 0.98867314]
F-score [0.97991584 0.08477869]
Support [335007    618]
**************************************
Iteration number   474
Class weights  {0: 1, 1: 474}
Performance on validation data - Confusion matrix
[[321775  13232]
 [     7    611]]
Precision, Recall, F-score, Support on validation data
Precision [0.99997825 0.04413783]
Recall [0.96050232 0.98867314]
F-score [0.97984284 0.08450315]
Support [335007    618]
*********************************

Precision, Recall, F-score, Support on validation data
Precision [0.99997817 0.04093528]
Recall [0.95726955 0.98867314]
F-score [0.97815789 0.07861554]
Support [335007    618]
**************************************
Iteration number   496
Class weights  {0: 1, 1: 496}
Performance on validation data - Confusion matrix
[[320637  14370]
 [     7    611]]
Precision, Recall, F-score, Support on validation data
Precision [0.99997817 0.04078499]
Recall [0.95710537 0.98867314]
F-score [0.97807218 0.07833836]
Support [335007    618]
**************************************
Iteration number   497
Class weights  {0: 1, 1: 497}
Performance on validation data - Confusion matrix
[[320598  14409]
 [     7    611]]
Precision, Recall, F-score, Support on validation data
Precision [0.99997817 0.04067909]
Recall [0.95698896 0.98867314]
F-score [0.97801138 0.07814299]
Support [335007    618]
**************************************
Iteration number   498
Class weights  {0: 1, 1: 498}
Performance on validation 